In [2]:
import sys
sys.path.append('../zoho_data_app')

import main_module as m

import pandas as pd
import gspread
import re
import datetime

import dateutil

%load_ext autoreload
%autoreload 2

gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID_SOURCE = '1PiRLPCroErrgZ07dlLdZZxUKzFmSZNnFwLy48MDmK2A'

execeptional_list = ['Database Name', 'Reciprocal']


In [173]:
# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID_SOURCE)

# Get all sheet names
sheet_names = [sheet.title for sheet in spreadsheet_source.worksheets()]
sheet_names_id = [sheet.id for sheet in spreadsheet_source.worksheets()]
print(sheet_names)

['Database Name', 'HOK', 'KC 2021', 'KCB-Elite-ENG', 'KCB-ID-Excluded KCID', 'KCB-NH-ID', 'KCB-NH-ENG', 'KCB-ENG-Excluded KCID', 'KCB-St.Martin', 'KCE', 'KCI-Elite-Non India', 'KCI-India', 'KCI-NH-India', 'KCI-NH-Non India', 'KCI-Non India', 'KCID-ID', 'KCID-ENG', 'Reciprocal']


In [174]:
len(sheet_names)

18

In [175]:

count = 0
frames = []

while count < len(sheet_names):
    if sheet_names[count] not in execeptional_list:
        print(f"Use this list {sheet_names[count]}")
        sheet = spreadsheet_source.worksheet(sheet_names[count])

        globals()['worksheet%s' % count] = spreadsheet_source.worksheet(sheet_names[count])
        globals()['rows%s' % count] = globals()['worksheet%s' % count].get_all_records()

        globals()['df%s' % count] = pd.DataFrame(globals()['rows%s' % count])

        globals()['df%s' % count]['Sheet Name'] = sheet_names[count]

        frames.append(globals()['df%s' % count])
        # print(f"Sheet {sheet_names[count]} has {sheet.row_count} rows and {sheet.col_count} columns")
        count += 1
    else:
        print(f"Not Use This List {sheet_names[count]}")

        count += 1
    
result = pd.concat(frames, ignore_index=True)
result
                    

Not Use This List Database Name
Use this list HOK
Use this list KC 2021
Use this list KCB-Elite-ENG
Use this list KCB-ID-Excluded KCID
Use this list KCB-NH-ID
Use this list KCB-NH-ENG
Use this list KCB-ENG-Excluded KCID
Use this list KCB-St.Martin
Use this list KCE
Use this list KCI-Elite-Non India
Use this list KCI-India
Use this list KCI-NH-India
Use this list KCI-NH-Non India
Use this list KCI-Non India
Use this list KCID-ID
Use this list KCID-ENG
Not Use This List Reciprocal


,number,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town,City,Country,Post Code,...,WeddingDate,Account Type,Account Status,Servicing Office,Sheet Name,Keywords,PanCardNo,OdysseyNo,segment,Account Currency
0,1118668,1109047,Elliot,Styche,,,Sydney,,Australia,,...,,HOUSE OF KARMA,Active,KARMA CONCIERGE BALI,HOK,NaN,NaN,NaN,NaN,NaN
1,1118840,1109358,Marcus,Willis,"8/79 Oxford St,",COLLINGWOOD,MELBOURNE,VICTORIA,Australia,3066,...,,HOUSE OF KARMA,Active,KARMA CONCIERGE BALI,HOK,NaN,NaN,NaN,NaN,NaN
2,1118843,1109361,Jackson,Springer,,,MELBOURNE,VICTORIA,Australia,,...,,HOUSE OF KARMA,Active,KARMA CONCIERGE BALI,HOK,NaN,NaN,NaN,NaN,NaN
3,1118885,1109425,Kirk,Mathews,,,SYDNEY,NEW SOUTH WALES,Australia,,...,,HOUSE OF KARMA,Active,KARMA CONCIERGE BALI,HOK,NaN,NaN,NaN,NaN,NaN
4,1118886,1109427,Alexander,Parr,,,SYDNEY,NEW SOUTH WALES,Australia,,...,,HOUSE OF KARMA,Active,KARMA CONCIERGE BALI,HOK,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80777,64829,70245,Denis,Buyok,"UNITS 12 & 13, BLOK A REGENT SQUARE",SIMPANG 150,KAMPONG KIARONG,.,Brunei,BE1318,...,,KARMA CLUB,Active,KARMA CONCIERGE BALI,KCID-ENG,NaN,NaN,C82001492,NaN,NaN
80778,66328,71911,Adrian,Tang,JL MOHAMAD KAFI 1 BLOK A NO 12 C,"TAMAN BOTANIK, JAGAKARSA",JAKARTA,DKI JAKARTA,Indonesia,12620,...,,KARMA CLUB,Active,KARMA CONCIERGE BALI,KCID-ENG,NaN,NaN,C82013297,NaN,NaN
80779,66328,198495,Bianca,Sugiarto,.,"TAMAN BOTANIK, JAGAKARSA",.,.,Indonesia,.,...,,KARMA CLUB,Active,KARMA CONCIERGE BALI,KCID-ENG,NaN,NaN,C82013297,NaN,NaN
80780,945865,948906,Nicole Audreylia Vanessa,Sugiyanto,-,-,-,-,Australia,-,...,,KARMA CLUB,Active,KARMA CONCIERGE BALI,KCID-ENG,NaN,NaN,C82043525,NaN,NaN


In [176]:
result.columns

Index(['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
       'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office',
       'Sheet Name', 'Keywords', 'PanCardNo', 'OdysseyNo', 'segment',
       'Account Currency'],
      dtype='object')

In [177]:
result['Main DB'] = ''
result['Sub DB'] = ''

result.loc[result['Sheet Name'] == 'HOK', 'Main DB'] = 'HOK'
result.loc[result['Sheet Name'] == 'KC 2021', ['Main DB','Sub DB']] = ['KC 2021','KCEU']
result.loc[result['Sheet Name'] == 'KCB-Elite-ENG', ['Main DB','Sub DB']] = ['KCB', 'Elite-Eng']
result.loc[result['Sheet Name'] == 'KCB-ID-Excluded KCID', ['Main DB','Sub DB']] = ['KCB', 'ID-Exc']
result.loc[result['Sheet Name'] == 'KCB-NH-ID', ['Main DB','Sub DB']] = ['KCB', 'ID-NH']
result.loc[result['Sheet Name'] == 'KCB-NH-ENG', ['Main DB','Sub DB']] = ['KCB', 'ENG-NH']
result.loc[result['Sheet Name'] == 'KCB-ENG-Excluded KCID', ['Main DB','Sub DB']] = ['KCB', 'ENG-Exc']
result.loc[result['Sheet Name'] == 'KCB-St.Martin', ['Main DB','Sub DB']] = ['KCB', 'ST.Martin-Elite']
result.loc[result['Sheet Name'] == 'KCE', ['Main DB']] = ['KCDEU']
result.loc[result['Sheet Name'] == 'KCI-Elite-Non India', ['Main DB','Sub DB']] = ['KCI', 'Elite-Non India']
result.loc[result['Sheet Name'] == 'KCI-India', ['Main DB','Sub DB']] = ['KCI', 'India']
result.loc[result['Sheet Name'] == 'KCI-NH-India', ['Main DB','Sub DB']] = ['KCI', 'NH-India']
result.loc[result['Sheet Name'] == 'KCI-NH-Non India', ['Main DB','Sub DB']] = ['KCI', 'NH-Non India']
result.loc[result['Sheet Name'] == 'KCI-Non India', ['Main DB','Sub DB']] = ['KCI', 'Non India']
result.loc[result['Sheet Name'] == 'KCID-ID', ['Main DB','Sub DB']] = ['KCID', 'ID']
result.loc[result['Sheet Name'] == 'KCID-ENG', ['Main DB','Sub DB']] = ['KCID', 'ENG']

In [178]:
current_date = datetime.date.today()

formatted_date = current_date.strftime("%d-%m-%Y")
result['Date DB'] = formatted_date

result['Tags'] = 'Members'

In [179]:
# result.columns
# result.columns = ['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
#        'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
#        'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
#        'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office', 
#        'Keywords', 'PanCardNo', 'OdysseyNo', 'segment',
#        'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags']

In [180]:
result.rename(columns={
    'segment': 'Member Segment',
}, inplace=True)

In [181]:
result.fillna('', inplace=True)

In [182]:
result['Email Old'] = result['Email']

m.clean_space(result, 'Email')
m.email_correction(result, 'Email')

result['Email Correction'] = 'No'
result.loc[result['Email Old']!=result['Email'], 'Email Correction'] = 'Yes'

c:\Users\fajar\Documents\Python\Task Offline\request_app\Karma Curated Event\main_module.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar\Documents\Python\Task Offline\request_app\Karma Curated Event\main_module.py:193: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate objec

In [183]:
result[['Email','Main DB']].loc[result['Email'].str.contains(',', na=False)]

,Email,Main DB
3320,"georganita@hotmail.com,",KCB
5995,"neil.aitchison@karmagroup.com,447947236783,",KCB
23598,"pthnatry@yahoo.com,vijetha.shastry@gmail.com",KCI
23804,"angelaoliveira399@gmail.com,florencedsouza2012...",KCI
23942,"ajaygupt70@gmail.com,parulgupt70@gmail.com",KCI
...,...,...
77109,"harpreet.panesar@icaew.com,robin_jhita@hotmail...",KCI
77150,"dcsoman@aol.com,dilip1soman@gmail.com",KCI
77382,"mick.yeomans@scotts.com,mickngeraldine@uwclub.net",KCI
77443,"kulwantsidhu1962@gmail.com,Sidhu.suky1@gmail.com",KCI


In [184]:
result.columns

Index(['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
       'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office',
       'Sheet Name', 'Keywords', 'PanCardNo', 'OdysseyNo', 'Member Segment',
       'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags', 'Email Old',
       'Email Correction'],
      dtype='object')

In [185]:
result = result.reindex(columns=['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email Old','Email',
       'Email Correction','Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office',
       'Sheet Name', 'Keywords', 'PanCardNo', 'OdysseyNo', 'Member Segment',
       'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags'])

In [186]:
result.to_excel(r'C:\Users\fajar\Documents\Python\Data\compiled-member.xlsx', index=False)

In [187]:
result.to_csv(r'C:\Users\fajar\Documents\Python\Data\compiled-member.csv', index=False)